In [ ]:
#NB: This code has been deprecated in favor of the next_weeks_model script.

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
#Parameters
file = "../tables/2021AP.xlsx"

next_week = 9

#Chose features to drop
feature_drop = ['Movement', 'Team']

#Set max rank hyperparameter
max_rank = 26

#Read in df
df = pd.read_excel(file)
df

,Team,Week,Rank,W,L,Winning Percentage,Opp. Rank,Opp. P5,Home,Result,Points Scored,Points Against,Margin,Next Week Rank,Movement
0,Alabama,1,1,0,0,1,14,1,0,W,44,13,31,1,0
1,Oklahoma,1,2,0,0,1,26,0,1,W,40,35,5,4,-2
2,Clemson,1,3,0,0,1,5,1,0,L,3,10,-7,6,-3
3,Ohio State,1,4,0,0,1,26,1,0,W,45,31,14,3,1
4,Georgia,1,5,0,0,1,3,1,0,W,10,3,7,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Coastal Carolina,11,22,8,1,0.888889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,Penn State,11,23,6,3,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,Louisiana-Lafayette,11,24,8,1,0.888889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,Pittsburgh,11,25,7,2,0.777778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Drop footer
df.drop(index = (df.index.stop - 1), inplace = True)

#Drop overflow
for index, row in df.iterrows():
    if row['Week'] >= next_week:
        df.drop(index = index, inplace = True)

In [4]:
#Encode Result column
for index, row in df.iterrows():
    if row['Result'] == 'W':
        df.at[index, 'Result'] = 1
    else: 
        df.at[index, 'Result'] = 0

In [5]:
#Get next week teams list
next_teams = df['Team'].loc[df['Week'] == next_week - 1]
next_rank_act = df['Next Week Rank'].loc[df['Week'] == next_week - 1]

#Get byes
byes_index = []
for index, row in df.iterrows():
    if row['Week'] == next_week - 1 and not(row['Opp. Rank'] >= 0):
        byes_index.append(index % 25)
        
byes_index

[0, 8, 10, 14, 18, 19]

In [6]:
#Drop null rows (teams with byes)
df.dropna(inplace = True)

In [7]:
#Drop irrelevant cols
df = df.drop(columns = feature_drop)

In [8]:
#Convert cols to num
for name, values in df.iteritems():
    df[name] = pd.to_numeric(values)

In [9]:
df.dtypes

Week                    int64
Rank                    int64
W                       int64
L                       int64
Winning Percentage    float64
Opp. Rank               int64
Opp. P5                 int64
Home                    int64
Result                  int64
Points Scored           int64
Points Against          int64
Margin                  int64
Next Week Rank          int64
dtype: object

In [10]:
#Set training data to Week < next week
train_df = df.loc[df['Week'] < next_week - 1]
X_train = train_df.drop(columns = ['Next Week Rank'])
y_train = train_df['Next Week Rank']

In [11]:
#Set testing data to Week = next week
test_df = df.loc[df['Week'] == next_week - 1]
X_test = test_df.drop(columns = ['Next Week Rank'])
y_test = test_df['Next Week Rank']

In [12]:
#Multiple Linear Regression
lin_model = LinearRegression().fit(X_train, y_train)
lin_model_output = lin_model.predict(X_test)

In [13]:
#Add back teams with byes to output
next_rank_pred = lin_model_output.tolist()
for item in byes_index:
    next_rank_pred.insert(item, 'bye')

In [14]:
#Build output col for pred rank
model_output = lin_model_output.tolist()

for item in byes_index:
    model_output.insert(item + 1, item + 1)
    
model_output.sort()    

sorted_output = model_output

In [15]:
#Build output df
df2 = pd.DataFrame()
df2['Predicted Rank'] = sorted_output
df2['Predicted Ordinal Rank'] = range(1, 26)

In [16]:
results = pd.DataFrame()
results['Team'] = next_teams
results['Predicted Rank'] = next_rank_pred
results['Actual Rank'] = next_rank_act

results.reset_index(inplace = True, drop = True)

#Predict that teams with byes don't change rank
for index, row in results.iterrows():
    if row['Predicted Rank'] == 'bye':
        results.at[index, 'Predicted Rank'] = index + 1

results['Predicted Rank'] = pd.to_numeric(results['Predicted Rank'], errors = 'coerce')
results['Actual Rank'] = pd.to_numeric(results['Actual Rank'])
results['Previous Rank'] = range(1, 26)

merged_results = results.merge(df2, how = 'left', left_on = 'Predicted Rank', right_on = 'Predicted Rank')

In [17]:
apor_col = []

for index, row in merged_results.iterrows():
    if row['Predicted Rank'] > 25:
        apor_col.append(max_rank)
    
    else:
        apor_col.append(row['Predicted Ordinal Rank'])

In [18]:
merged_results['Adj. Predicted Ordinal Rank'] = apor_col

In [19]:
merged_results = merged_results[['Team', 'Previous Rank', 'Predicted Rank', 'Predicted Ordinal Rank', 'Adj. Predicted Ordinal Rank', 'Actual Rank']]
merged_results

,Team,Previous Rank,Predicted Rank,Predicted Ordinal Rank,Adj. Predicted Ordinal Rank,Actual Rank
0,Georgia,1,1.000000,1,1,1
1,Cincinnati,2,3.996674,4,4,2
2,Oklahoma,3,2.509012,2,2,4
3,Alabama,4,3.880082,3,3,3
4,Ohio State,5,4.028894,5,5,5
5,Michigan,6,5.180630,6,6,6
6,Penn State,7,17.368500,17,17,20
7,Oklahoma State,8,16.548258,15,15,15
8,Michigan State,9,9.000000,7,7,8
9,Oregon,10,10.463553,8,8,7


In [20]:
import sys
sys.path.append('../libraries')

from ordinal.ordinal import get_error

In [21]:
r2_score(merged_results['Adj. Predicted Ordinal Rank'], merged_results['Actual Rank'])

0.9213195011073552

In [22]:
get_error(merged_results['Adj. Predicted Ordinal Rank'], merged_results['Actual Rank'], mode = 'total')

32

In [23]:
#merged_results.to_csv(f'week{next_week}_predictions.csv', header = True, index = False)